<a href="https://colab.research.google.com/github/Shaik1903/Vision-Transformer-from-scratch-ML-Project-/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import os
from PIL import Image
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data_path = "/content/drive/MyDrive/ML Project/mini/small"
output_path = "/content/drive/MyDrive/ML Project/mini/output"

In [5]:
# resizing the images and transfroming them into tensors
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.Grayscale(),
    transforms.ToTensor(),
])

In [6]:
# Loading the dataset
dataset = ImageFolder(root=data_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:
desired_dim = 512  # Adjust the desired dimension according to your requirements
linear_projection = torch.nn.Linear(256 * 256, desired_dim)

In [16]:
# Initialize a list to store the projected patches
all_projected_patches = []

In [17]:
for phase in ["train", "test", "val"]:
    phase_path = os.path.join(data_path, phase)
    for label in os.listdir(phase_path):
        label_path = os.path.join(phase_path, label)
        for root, _, files in os.walk(label_path):
            for file in files:
                if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                    image_path = os.path.join(root, file)
                    image = Image.open(image_path)
                    image = transform(image)
                    b, h, w = image.size()
                    flattened_patches = image.view(b, -1)
                    projected_patches = linear_projection(flattened_patches)
                    # Add the projected patches to the list
                    all_projected_patches.append(projected_patches)
# Convert the list to a torch tensor
all_projected_patches = torch.stack(all_projected_patches)

# Save the projected patches as a .pt file
output_file_path = os.path.join(output_path, "projected_patches.pt")
torch.save(all_projected_patches, output_file_path)

print(f"Projected patches saved at {output_file_path}")

Projected patches saved at /content/drive/MyDrive/ML Project/mini/output/projected_patches.pt


In [18]:
saved_file_path = "/content/drive/MyDrive/ML Project/mini/output/projected_patches.pt"

In [19]:
# Load the projected patches
loaded_data = torch.load(saved_file_path)

In [20]:
# Print the shape of the loaded data
print("Loaded data shape:", loaded_data.shape)

Loaded data shape: torch.Size([3002, 1, 512])
